In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from scipy.sparse import hstack
import cPickle as pickle

In [2]:
df = pd.concat([
        pd.concat([pd.read_pickle('../data/train_VAR_0404.p'), 
                   pd.read_pickle('../data/train_VAR_0493.p'),], axis=1),
        pd.concat([pd.read_pickle('../data/test_VAR_0404.p'), 
                   pd.read_pickle('../data/test_VAR_0493.p'),], axis=1)])

df.fillna(0, inplace=True)
df.replace('-1', '', inplace=True)
# df

In [3]:
vectorizer = CountVectorizer(min_df=4,
    stop_words=['a', 'and', 'for', 'of', 'to'])

combo = df['VAR_0404'].astype(str) + ' ' + df['VAR_0493'].astype(str)
counts_vec = vectorizer.fit_transform(combo)
print counts_vec.shape

# I would store the sparse counts vec in a pandas sparse df, 
# except pandas support for sparse sucks

for col in ['VAR_0404', 'VAR_0493']:
    df[col+'_bool'] = df[col] != ''

occ_feats = hstack([np.array(df[col+'_bool'])[:, None]
                for col in ['VAR_0404', 'VAR_0493']] + [counts_vec])
np.save('../data/occ_feats.npy', occ_feats)
# NOTE: The index 'ID' needs book-keeping when referring to this sparse count matrix

# Should re-order df index 'ID' before this whole process

(290463, 730)


# Scratchspace

In [4]:
q = []
for k, v in vectorizer.vocabulary_.items():
    q.append([k, counts_vec[:, v].sum()])
    
pd.DataFrame(q).sort(columns=1)

,0,1
364,phys,4
241,deve,4
231,car,4
230,graphic,4
628,emblamer,4
207,ophthalmologist,4
535,osteopathy,4
534,associates,4
188,facilitator,4
450,dispenser,4


In [5]:
df.iloc[list(set(df['VAR_0404'].nonzero()[0]).intersection(set(df['VAR_0493'].nonzero()[0])))]

,VAR_0404,VAR_0493,VAR_0404_bool,VAR_0493_bool
ID,,,,
2,CHIEF EXECUTIVE OFFICER,COMMUNITY ASSOCIATION MANAGER,True,True
168253,INDIVIDUAL - SOLE OWNER,ELECTRICIAN,True,True
233890,CONTACT,REGISTERED ASSOCIATE SOCIAL WORKER,True,True
130966,DIRECTOR,CERTIFIED RESPIRATORY THERAPIST,True,True
130968,PHARMACIST IN CHARGE,PHARMACIST,True,True
119185,ACCOUNTS RECEIVABLE,HOME HEALTH AIDE,True,True
32179,PRESIDENT,CERTIFIED BUILDING CONTRACTOR,True,True
73030,CEO,CERTIFIED NURSING ASSISTANT,True,True
262140,MEMBER,MANUFACTURING ENGINEER,True,True


In [12]:
# Making sure that the indices are ordered as I would expect

train_ids = pd.read_pickle('../data/train_VAR_0404.p').index
test_ids = pd.read_pickle('../data/test_VAR_0404.p').index
all_ids = df.index

all(all_ids == np.concatenate([train_ids, test_ids]))

True